### Run this in Python kernel

To reset font style of output (in case it is not the default one)

In [1]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            ;;[clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojure.edn :as edn]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative path to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Get list of tickers with ROE

In [3]:
;; Use merged database CRSP + COMPUSTAT
;; Compute the latest quarter ROE by using: niq/(share price * cshoq) in the merged dataset

;; parameters:
;; - file to dataset
;; - date (for getting the ROE list)

;; output:
;; {:data [{:tic AAPL, :year 1980, :ROE 0.002731} {...}], :ROE-set [0.002731 -0.04637 ...]}

;; need to use lazy sequence if applying on bigger dataset

(defn get-set-roe
    "return a set of maps of tickers and datadate"
    [file date]
    
    (loop [remaining file
            result-set []
            ROE-list [] ] ;; loop over the dataset
        
        (if (empty? remaining) ;; terminate when remaining dataset is empty
            {:data result-set :ROE-set ROE-list}
            
            (let [first-line (first remaining) ;; get first line
                next-remaining (rest remaining)
                [year month day] (map parse-int (str/split date #"-"))]

                (if (= date (get first-line :datadate)) 
                    (let [[niq PRC cshoq] (map edn/read-string [(get first-line :niq)(get first-line :PRC)(get first-line :cshoq)])
                          ROE (/ niq (* PRC cshoq))]
                        (recur next-remaining (conj result-set {:tic (get first-line :tic) :year year :ROE ROE}) (conj ROE-list ROE))
                    )
                    (recur next-remaining result-set ROE-list)
                )  
            )
        )
    )
)

#'clojure-backtesting.demo/get-set-roe

In [5]:
;; demo
(def data-demo (get-set-roe (read-csv-row "../resources/data-testing-merged.csv") "1980-12-18"))
(println data-demo) ;

{:data [{:tic AAPL, :year 1980, :ROE 0.002731506525434953} {:tic GM, :year 1980, :ROE -0.046370846428037134} {:tic IBM, :year 1980, :ROE 0.0238072629827958}], :ROE-set [0.002731506525434953 -0.046370846428037134 0.0238072629827958]}


In [6]:
;; fn to get a set of ROE

(defn get-ROE
 "return a set of ROE"
 [dataset]
 (into #{} (get dataset :ROE-set))
)

#'clojure-backtesting.demo/get-ROE

In [7]:
(defn get-roe-20
    "return a set tickers with ROE >= cut-off (i.e. roe-20)" ;;{"AAPL" "GM"}
    [data roe-20]
    (loop [remaining (get data :data)
            result-set []]
        (if (empty? remaining)
            (into #{} result-set)
            (let [first-line (first remaining)
                next-remaining (rest remaining)]
                (if (>= (get first-line :ROE) roe-20)
                    (recur next-remaining (conj result-set (get first-line :tic)))           
                    (recur next-remaining result-set)            
                )  
            )
        )
    )
)

#'clojure-backtesting.demo/get-roe-20

In [9]:
;; test examples

;; get a set of ROE
(def roe-list-demo (get-ROE data-demo))
(println roe-list-demo)

;; get a sorted set of ROE
(def roe-sorted-set (apply sorted-set roe-list-demo))
(println roe-sorted-set)

;; get a sorted vector of ROE
(def roe-sorted-vec (into '[] roe-sorted-set))
(println roe-sorted-vec)

;; find the 20th percentile ROE
(def roe-20-demo (nth roe-sorted-vec (int (* 0.8 (count roe-sorted-vec)))))
(println roe-20-demo)
         
;; get the list of tickers to buy
(def stock-to-buy-test (get-roe-20 data-demo roe-20-demo))
(println stock-to-buy-test)
(println (type stock-to-buy-test)) ;;stored in a set format

#{0.0238072629827958 -0.046370846428037134 0.002731506525434953}
#{-0.046370846428037134 0.002731506525434953 0.0238072629827958}
[-0.046370846428037134 0.002731506525434953 0.0238072629827958]
0.0238072629827958
#{IBM}
clojure.lang.PersistentHashSet


nil

### Initialise portfolio

In [10]:
(init-portfolio "1980-12-18" 100000)

#'clojure-backtesting.order/portfolio-value

In [11]:
(keys (deref available-tics))

("AAPL" "F" "IBM")

In [12]:
(def year-count 3) ;; hold the stock for 3 years
(def start-year 1980)
(def rebalance-md (subs (get-date) 4)) ; = -12-18

(def rebalance-years (into [] (range (+ start-year 1) (+ (+ start-year 1) year-count) 1))) ; rebalance every year

(def rebalance-dates []) ; [1981-12-18, 1982-12-18, 1983-12-18]
(doseq [year rebalance-years]
  (def rebalance-dates (conj rebalance-dates (str year rebalance-md)))
)

(def end-date (last rebalance-dates)) ; 1983-12-18

;; get stock tickers and ROE data
;; output {{:tic "AAPL" :year "1980" :ROE x.x}{...}{:ROE 1.x 2.x 3.x 10.x}}
(def stock-data (get-set-roe (read-csv-row "../resources/data-testing-merged.csv") "1980-12-18"))

;; sort the stocks according to their ROE (= Net Income/Total Equity)

;; get a set of ROE
(def roe-list (get-ROE stock-data))

;; sorting function, return tickers of the top 20% stocks
;; determine what are the top 20% stocks and turn the sorted-set into list format
(def roe-sorted (into '[] (apply sorted-set roe-list)))

;(println roe-list)
;(println (int (* 0.8 (count roe-sorted))))

;; find the 20% cut-off ROE value
(let [roe-20 (nth roe-sorted (int (* 0.8 (count roe-sorted))))]    
    ;;get the tickers of the top 20 with function get-roe-20
    (def stocks-to-buy (get-roe-20 stock-data roe-20))
)

;; buy the top 20% stocks and the sell the stocks that are not in the top 20% this year      
(def stocks-to-buy-list (into [] stocks-to-buy))

;; buy stocks 
(doseq [stock stocks-to-buy-list]        
  (order stock 10)
)

(update-eval-report (get-date)) ; update evaluation metrics
(next-date)

(while (not= (empty? rebalance-dates) true)
  ;(println (get-date)) ; debug
  (if (t/after? (t/local-date (get-date)) (t/local-date (first rebalance-dates))) ; check if (get-date) has passed first date in rebalance-dates
    (do
      (def rebalance-dates (rest rebalance-dates)) ; pop the first date in rebalance-dates
      ;(println (rest rebalance-dates)) ; debug
      (while (empty? (get-set-roe (read-csv-row "../resources/data-testing-merged.csv") (get-date)))
          (next-date) ;; move on to next date til data is not empty
      )
      (def stock-data (get-set-roe (read-csv-row "../resources/data-testing-merged.csv") (get-date)))
      (def roe-list (get-ROE stock-data))
      (def roe-sorted (into '[] (apply sorted-set roe-list)))
      (let [roe-20 (nth roe-sorted (int (* 0.8 (count roe-sorted))))]    
          (def stocks-to-buy (get-roe-20 stock-data roe-20))
      )
      (def stocks-to-buy-list (into [] stocks-to-buy))

      ;; sell stocks held in portfolio
      (doseq [[ticker row] (deref portfolio)]
        (if (not= ticker :cash)      
          (order ticker -10)
        )
      )

      ;; buy stocks
      (doseq [stock stocks-to-buy-list]       
        (order stock 10)
      )

      (update-eval-report (get-date)) ; update evaluation metrics
    )
  )
  (next-date) ; move on to the next trading day
)

;; sell stocks held in portfolio (if ticker != "cash" && quantity > 0)
(doseq [[ticker row] (deref portfolio)]
  (if (and (not= ticker :cash) (= (compare (get row :quantity) 0) 1))
      (order ticker -10)
  )
)

The order request 1982-01-08 | IBM | 0 fails.
Failure reason: No such date or ticker in the dataset or the dataset has reached the end.
The order request 1983-12-19 | GM | 10 fails.
Failure reason: No such date or ticker in the dataset or the dataset has reached the end.


nil

### Check order record

In [13]:
(pprint/print-table (deref order-record))


|      :date | :tic | :price |   :aprc | :quantity |
|------------+------+--------+---------+-----------|
| 1980-12-19 |  IBM | 64.625 |  736.07 |        10 |
| 1981-12-22 |  IBM | 56.875 |  714.58 |       -10 |
| 1981-12-22 |  IBM | 56.875 |  714.58 |        10 |
| 1982-12-21 |  IBM |  95.75 |  915.74 |       -10 |
| 1982-12-21 |  IBM |  95.75 |  915.74 |        10 |
| 1983-12-20 |  IBM | 121.75 | 1030.91 |       -10 |


nil

### Check portfolio record

In [14]:
;; view final portfolio
(view-portfolio)


| :asset | :price |   :aprc | :quantity | :tot-val |
|--------+--------+---------+-----------+----------|
|   cash |    N/A |     N/A |       N/A |   102948 |
|    IBM | 121.75 | 1030.91 |         0 |        0 |


nil

In [15]:
;; view portfolio value and return
(view-portfolio-record)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage |
|------------+------------+------------+----------+-------+-----------|
| 1980-12-18 |    $100000 |      0.00% |    0.00% | $0.00 |     0.00% |
| 1980-12-19 |    $100000 |      0.00% |    0.00% | $0.00 |     0.00% |
| 1980-12-22 |    $100122 |      0.12% |    0.12% | $0.00 |     0.00% |
| 1980-12-23 |    $100128 |      0.01% |    0.13% | $0.00 |     0.00% |
| 1980-12-24 |    $100188 |      0.06% |    0.19% | $0.00 |     0.00% |
| 1980-12-26 |    $100212 |      0.02% |    0.21% | $0.00 |     0.00% |
| 1980-12-29 |    $100164 |     -0.05% |    0.16% | $0.00 |     0.00% |
| 1980-12-30 |    $100194 |      0.03% |    0.19% | $0.00 |     0.00% |
| 1980-12-31 |    $100158 |     -0.04% |    0.16% | $0.00 |     0.00% |
| 1981-01-02 |    $100218 |      0.06% |    0.22% | $0.00 |     0.00% |
| 1981-01-05 |    $100271 |      0.05% |    0.27% | $0.00 |     0.00% |
| 1981-01-06 |    $100330 |      0.06% |    0.33% | $0.00 |    

| 1981-06-01 |     $99867 |      0.08% |   -0.13% | $0.00 |     0.00% |
| 1981-06-02 |     $99847 |     -0.02% |   -0.15% | $0.00 |     0.00% |
| 1981-06-03 |     $99808 |     -0.04% |   -0.19% | $0.00 |     0.00% |
| 1981-06-04 |     $99748 |     -0.06% |   -0.25% | $0.00 |     0.00% |
| 1981-06-05 |     $99768 |      0.02% |   -0.23% | $0.00 |     0.00% |
| 1981-06-08 |     $99762 |     -0.01% |   -0.24% | $0.00 |     0.00% |
| 1981-06-09 |     $99715 |     -0.05% |   -0.28% | $0.00 |     0.00% |
| 1981-06-10 |     $99735 |      0.02% |   -0.26% | $0.00 |     0.00% |
| 1981-06-11 |     $99801 |      0.07% |   -0.20% | $0.00 |     0.00% |
| 1981-06-12 |     $99821 |      0.02% |   -0.18% | $0.00 |     0.00% |
| 1981-06-15 |     $99847 |      0.03% |   -0.15% | $0.00 |     0.00% |
| 1981-06-16 |     $99795 |     -0.05% |   -0.20% | $0.00 |     0.00% |
| 1981-06-17 |     $99775 |     -0.02% |   -0.22% | $0.00 |     0.00% |
| 1981-06-18 |     $99728 |     -0.05% |   -0.27% | $0.00 |     

| 1981-11-10 |     $99526 |      0.04% |   -0.47% | $0.00 |     0.00% |
| 1981-11-11 |     $99540 |      0.01% |   -0.46% | $0.00 |     0.00% |
| 1981-11-12 |     $99555 |      0.01% |   -0.45% | $0.00 |     0.00% |
| 1981-11-13 |     $99490 |     -0.06% |   -0.51% | $0.00 |     0.00% |
| 1981-11-16 |     $99454 |     -0.04% |   -0.55% | $0.00 |     0.00% |
| 1981-11-17 |     $99454 |      0.00% |   -0.55% | $0.00 |     0.00% |
| 1981-11-18 |     $99418 |     -0.04% |   -0.58% | $0.00 |     0.00% |
| 1981-11-19 |     $99418 |      0.00% |   -0.58% | $0.00 |     0.00% |
| 1981-11-20 |     $99432 |      0.01% |   -0.57% | $0.00 |     0.00% |
| 1981-11-23 |     $99425 |     -0.01% |   -0.58% | $0.00 |     0.00% |
| 1981-11-24 |     $99526 |      0.10% |   -0.47% | $0.00 |     0.00% |
| 1981-11-25 |     $99548 |      0.02% |   -0.45% | $0.00 |     0.00% |
| 1981-11-27 |     $99653 |      0.11% |   -0.35% | $0.00 |     0.00% |
| 1981-11-30 |     $99653 |      0.00% |   -0.35% | $0.00 |     

| 1982-04-27 |    $100262 |     -0.04% |    0.26% | $0.00 |     0.00% |
| 1982-04-28 |    $100249 |     -0.01% |    0.25% | $0.00 |     0.00% |
| 1982-04-29 |    $100211 |     -0.04% |    0.21% | $0.00 |     0.00% |
| 1982-04-30 |    $100218 |      0.01% |    0.22% | $0.00 |     0.00% |
| 1982-05-03 |    $100230 |      0.01% |    0.23% | $0.00 |     0.00% |
| 1982-05-04 |    $100243 |      0.01% |    0.24% | $0.00 |     0.00% |
| 1982-05-05 |    $100262 |      0.02% |    0.26% | $0.00 |     0.00% |
| 1982-05-06 |    $100293 |      0.03% |    0.29% | $0.00 |     0.00% |
| 1982-05-07 |    $100287 |     -0.01% |    0.29% | $0.00 |     0.00% |
| 1982-05-10 |    $100242 |     -0.04% |    0.24% | $0.00 |     0.00% |
| 1982-05-11 |    $100267 |      0.03% |    0.27% | $0.00 |     0.00% |
| 1982-05-12 |    $100242 |     -0.03% |    0.24% | $0.00 |     0.00% |
| 1982-05-13 |    $100229 |     -0.01% |    0.23% | $0.00 |     0.00% |
| 1982-05-14 |    $100229 |      0.00% |    0.23% | $0.00 |     

| 1982-10-07 |    $101088 |      0.06% |    1.08% | $0.00 |     0.00% |
| 1982-10-08 |    $101123 |      0.03% |    1.12% | $0.00 |     0.00% |
| 1982-10-11 |    $101230 |      0.11% |    1.22% | $0.00 |     0.00% |
| 1982-10-12 |    $101196 |     -0.03% |    1.19% | $0.00 |     0.00% |
| 1982-10-13 |    $101230 |      0.03% |    1.22% | $0.00 |     0.00% |
| 1982-10-14 |    $101077 |     -0.15% |    1.07% | $0.00 |     0.00% |
| 1982-10-15 |    $101048 |     -0.03% |    1.04% | $0.00 |     0.00% |
| 1982-10-18 |    $101207 |      0.16% |    1.20% | $0.00 |     0.00% |
| 1982-10-19 |    $101168 |     -0.04% |    1.16% | $0.00 |     0.00% |
| 1982-10-20 |    $101274 |      0.10% |    1.27% | $0.00 |     0.00% |
| 1982-10-21 |    $101241 |     -0.03% |    1.23% | $0.00 |     0.00% |
| 1982-10-22 |    $101196 |     -0.04% |    1.19% | $0.00 |     0.00% |
| 1982-10-25 |    $101071 |     -0.12% |    1.07% | $0.00 |     0.00% |
| 1982-10-26 |    $101162 |      0.09% |    1.16% | $0.00 |     

| 1983-03-21 |    $102037 |      0.03% |    2.02% | $0.00 |     0.00% |
| 1983-03-22 |    $102012 |     -0.02% |    1.99% | $0.00 |     0.00% |
| 1983-03-23 |    $102093 |      0.08% |    2.07% | $0.00 |     0.00% |
| 1983-03-24 |    $102098 |      0.00% |    2.08% | $0.00 |     0.00% |
| 1983-03-25 |    $102093 |     -0.00% |    2.07% | $0.00 |     0.00% |
| 1983-03-28 |    $102103 |      0.01% |    2.08% | $0.00 |     0.00% |
| 1983-03-29 |    $102123 |      0.02% |    2.10% | $0.00 |     0.00% |
| 1983-03-30 |    $102178 |      0.05% |    2.15% | $0.00 |     0.00% |
| 1983-03-31 |    $102078 |     -0.10% |    2.06% | $0.00 |     0.00% |
| 1983-04-04 |    $102103 |      0.02% |    2.08% | $0.00 |     0.00% |
| 1983-04-05 |    $102088 |     -0.01% |    2.07% | $0.00 |     0.00% |
| 1983-04-06 |    $102113 |      0.02% |    2.09% | $0.00 |     0.00% |
| 1983-04-07 |    $102138 |      0.02% |    2.12% | $0.00 |     0.00% |
| 1983-04-08 |    $102163 |      0.02% |    2.14% | $0.00 |     

| 1983-08-31 |    $102831 |      0.08% |    2.79% | $0.00 |     0.00% |
| 1983-09-01 |    $102799 |     -0.03% |    2.76% | $0.00 |     0.00% |
| 1983-09-02 |    $102841 |      0.04% |    2.80% | $0.00 |     0.00% |
| 1983-09-06 |    $102919 |      0.08% |    2.88% | $0.00 |     0.00% |
| 1983-09-07 |    $102937 |      0.02% |    2.90% | $0.00 |     0.00% |
| 1983-09-08 |    $102933 |     -0.00% |    2.89% | $0.00 |     0.00% |
| 1983-09-09 |    $102933 |      0.00% |    2.89% | $0.00 |     0.00% |
| 1983-09-12 |    $102905 |     -0.03% |    2.86% | $0.00 |     0.00% |
| 1983-09-13 |    $102924 |      0.02% |    2.88% | $0.00 |     0.00% |
| 1983-09-14 |    $102956 |      0.03% |    2.91% | $0.00 |     0.00% |
| 1983-09-15 |    $102914 |     -0.04% |    2.87% | $0.00 |     0.00% |
| 1983-09-16 |    $102965 |      0.05% |    2.92% | $0.00 |     0.00% |
| 1983-09-19 |    $102987 |      0.02% |    2.94% | $0.00 |     0.00% |
| 1983-09-20 |    $102992 |      0.00% |    2.95% | $0.00 |     

nil

### Generate evaluation report

In [16]:
(eval-report)


|      :date | :pnl-pt | :sharpe | :tot-val |  :vol |
|------------+---------+---------+----------+-------|
| 1980-12-18 |      $0 |   0.00% |  $100000 | 0.00% |
| 1981-12-21 |    $-71 |  -5.04% |   $99785 | 0.04% |
| 1982-12-20 |    $359 |  36.79% |  $101796 | 0.05% |
| 1983-12-19 |    $491 |  57.40% |  $102948 | 0.05% |


nil

### Plot variables
Below are example codes that show how to plot different variables in the portfolio record / evaluation record.

### 1. Portfolio total value

In [17]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [18]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "total value")
  data))

#'clojure-backtesting.demo/data-to-plot

In [19]:
(first data-to-plot)

{:date "1980-12-18", :tot-value 100000, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0, :plot "total value"}

In [20]:
(plot data-to-plot :plot :date :tot-value false)

### 2. Plot sharpe ratio

In [21]:
(def data (deref eval-record))

; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "sharpe ratio")
  data))

(plot data-to-plot :plot :date :sharpe false)

### 3. Plot stock price

In [22]:
(def data (deref order-record))

#'clojure-backtesting.demo/data

In [23]:
(first data)

{:date "1980-12-19", :tic "IBM", :price 64.625, :aprc "736.07", :quantity 10}

In [24]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "price")
  data))

(plot data-to-plot :plot :date :price false)